In [1]:
import re
import graphviz

In [50]:
# COMMMENTS

# (1) : we assumed this pattern is valid : a*?  (https://regex101.com)
# (2) : ?? is invalid
# (3) : ++ is invalid
# (4) : ** is invalid

open_list = ["[","("]
close_list = ["]",")"]
# Function to check parentheses
def check_braces(myStr):
    stack = []
    for i in myStr:
        if i in open_list:
            stack.append(i)
        elif i in close_list:
            pos = close_list.index(i)
            if ((len(stack) > 0) and
                (open_list[pos] == stack[len(stack)-1])):
                stack.pop()
            else:
                return False
    if len(stack) == 0:
        return True
    else:
        return False

def check_ORs(regex):

  if(regex.find('|)') != -1 or regex.find('(|') != -1):
    return False

  ORs = regex.count('|')
  splitted = list(filter(lambda x: x, regex.split('|')))
  if(len(splitted) != ORs + 1):
    return False
  return True

def check_valid_question_mark(regex):
  matches = re.findall(r'\?{2,}', regex)
  if(len(matches) > 0):
    return False

  if(regex[0] == '?'):
    return False

  if(regex.find('|?') != -1):
    return False

  return regex

def check_valid_plus(regex):
  matches = re.findall(r'\+{2,}', regex)
  if(len(matches) > 0):
    return False

  if(regex[0] == '+'):
    return False

  if(regex.find('|+') != -1):
    return False

  return regex

def check_valid_star(regex):
  matches = re.findall(r'\*{2,}', regex)
  if(len(matches) > 0):
    return False

  if(regex[0] == '*'):
    return False

  if(regex.find('|*') != -1):
    return False

  return True


def validate_regex(regex):

  # trailing spaces for regex
  regex = regex.strip()

  # check ORs (|)
  if (not check_ORs(regex)):
    return False

  # check braces
  if(not check_braces(regex)):
    return False

  # check ?
  regex = check_valid_question_mark(regex)
  if(not regex):
    return False

  # check +
  regex = check_valid_plus(regex)
  if(not regex):
    return False

  # check *
  if(not check_valid_star(regex)):
    return False

  pettern = "[a-zA-Z0-9\.\+\?\*\[\]\(\)\-\|]"
  matched = re.findall(pettern, regex)
  if(len(matched) != len(regex)):
    return False

  return True

In [3]:
# using built-in re.compile(regex)
def validateRegex(regex):
    try:
        re.compile(regex)
    except re.error:
        print(f"Invalid input: {regex}")
        return False
    return True

In [4]:
class postfixByShuntYard:
    def __init__(self, regex):
        self.regex = regex
        self.postfix = self.applyShuntYard(regex)

    def getPostfixedRegex(self):
        return self.postfix

    def applyShuntYard(self, regex):
        # *, +, ? : have the same precendence level
        # . : mid precendence level
        # | : lowest precedence level
        symbolPrecedence = {'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}
        postfix = ""
        stack = []
        # If we find [xyz] as example, this would be converted to (x|y|z)
        for i in range(len(regex)):
            if regex[i] == '[':
                j = i + 1
                while regex[j] != ']':
                    if regex[j].isalnum() and regex[j + 1].isalnum():
                        regex = regex[:j + 1] + '|' + regex[j + 1:]
                    j += 1

        # Replace the [] with ()
        regex = regex.replace('[', '(')
        regex = regex.replace(']', ')')

        # print("step 1: ", regex)

        # If we find a-f would be converted to (a|b|c|d|e|f)
        for i in range(regex.count('-')):
            for j in range(len(regex)):
                character = regex[j]
                if character == '-':
                    end = regex[j + 1]
                    start = regex[j - 1]
                    range_list = ''
                    for k in range(int(ord(end) - ord(start))):
                        range_list = range_list + '|'
                        range_list = range_list + chr(ord(start) + k + 1)
                    regex = regex[0: j] + range_list + regex[j + 2:]
                    break

        # print("step 2: ", regex)

        # We use . to divide the regex to subRegex
        # example ab(b|c)*d+ will be converted to ab.bc|*.d+.
        dotLocations = []
        for i in range(len(regex) - 1):
            startSymbols = [')', "*","+", "*"]
            endSymbols = ["*", "+", ".", "|", ")"]
            if regex[i] in startSymbols and regex[i+1] not in endSymbols:
                dotLocations.append(i)
            elif regex[i].isalnum() and (regex[i+1].isalnum() or regex[i+1] == '('):
                dotLocations.append(i)

        for i in range(len(dotLocations)):
            regex = regex[:dotLocations[i] + 1 + i] + '.' + regex[dotLocations[i] + 1 + i:]

        # print("step 3: ", regex)

        # Applying Shunting Yard Algorithm
        for i in range(len(regex)):
            character = regex[i]
            # If ( -> push in stack
            if character == '(':
                stack.append(character)
            # If ) -> pop from stack until ( is found
            elif character == ')':
                while stack[-1] != '(':
                    postfix = postfix + stack[-1]
                    stack.pop()
                stack.pop()  # to remove ( from the stack

            # order the operators based on their precedence
            elif character in symbolPrecedence:
                while stack and symbolPrecedence.get(character, 0) <= symbolPrecedence.get(stack[-1], 0):
                    postfix = postfix + stack[-1]
                    stack.pop()
                stack.append(character)

            # If a normal character (not an symbol or parenthesis)
            else:
                postfix = postfix + character
        # pop any remaining symbols from the stack and appends them to the postfix string
        while stack:
            postfix = postfix + stack[-1]
            stack.pop()

        # print("final step: ", regex)
        return postfix

In [5]:
class State:
    def __init__(self, name, transitions=[], parents=[], start=False, accepted=True):
        self.name = name
        self.parents = []
        self.transitions = []
        self.start = start
        self.accepted = accepted

    def transition(self, symbol, state):
        self.transitions.append((symbol, state))
        self.accepted = False
        state.parents.append(self)

    def parents(self):
        return self.parents.copy()

In [6]:
def visualize(file_name, states, view=False):

    graph = graphviz.Digraph(engine='dot')

    for state, transitions in states.items():
        if state == 'startingState':
            continue
        if transitions['isAcceptedState']:
            graph.node(state, shape='doublecircle')
        else:
            graph.node(state, shape='circle')

        for char, next_state in transitions.items():

            if char == 'isAcceptedState':
                continue
            children_states = next_state.split(',')

            for child in children_states:
                graph.edge(state, child, label=char)
    graph.render(file_name, view=view)

In [7]:
class NFA:
    def __init__(self, postfix=None, start=None, accept=None):
        self.start = start
        self.accept = accept
        if postfix and not start and not accept:
            obj = self.convertToNFA(postfix)
            self.start = obj.start
            self.accept = obj.accept

    def getStateByName(self, name):
        for state in self.getStates():
            if state.name == name:
                return state
        return None

    def getStatesByNames(self, names):
        names = names.split()
        result = []
        for name in names:
            result.append(self.getStateByName(name))
        return result

    def isAcceptedState(self, states):
        for state in states:
            if state.accepted:
                return True
        return False

    def getStates(self):
        visited = set()
        states = []
        queue = [self.start]
        visited.add(self.start)
        while queue:
            state = queue.pop(0)
            states.append(state)
            for (transition) in state.transitions:
                if transition[1] not in visited:
                    visited.add(transition[1])
                    queue.append(transition[1])

        return states

    def getInputSymbols(self):
        states = self.getStates()
        symbols = set()
        for state in states:
            for symbol, next_state in state.transitions:
                if symbol != 'ϵ':
                    symbols.add(symbol)
        return list(symbols)

    def convertToNFA(self, postfix):
        stack = []
        stateCounter = 0
        for character in postfix:
            if character == '.':
                # state_1 is the old state in the stack
                # state_2 is the new state in the stack
                state_2 = stack.pop()
                state_1 = stack.pop()
                state_1.accept.transition('ϵ', state_2.start)
                stack.append(NFA(start = state_1.start, accept = state_2.accept))

            elif character == '*':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', accept)
                state_1.accept.transition('ϵ', start)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '|':
                state_2 = stack.pop()
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', state_2.start)
                state_1.accept.transition('ϵ', accept)
                state_2.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '+':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                state_1.accept.transition('ϵ', start)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '?':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', accept)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            else:
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition(character, accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

        return stack.pop()

    def JSONIFY(self):
        states = {}
        for state in self.getStates():
            stateObj = {
                'isAcceptedState': state.accepted,
            }
            for character, nextState in state.transitions:
                if character not in stateObj:   # if one transition
                    stateObj[character] = nextState.name
                else:   # if multiple transitions to the same state
                    stateObj[character] += ',' + nextState.name
            states[state.name] = stateObj

        return {
            'startingState': self.start.name,
            **states,
        }

    def visualize(self, name, view=False):
        json = self.JSONIFY()
        graph = graphviz.Digraph(engine='dot')
        for state, transitions in json.items():
            if state == 'startingState':
                continue
            if transitions['isAcceptedState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')
            for char, nextState in transitions.items():
                if char == 'isAcceptedState':
                    continue
                children = nextState.split(',')
                for child in children:
                    graph.edge(state, child, label=char)
        graph.render(name, view=view)
        return graph



In [8]:
class DFA:
    def __init__(self, nfa):
        self.nfa = nfa
        self.states = self.convertNFAtoDFA(nfa)

    def getStateByName(self, name):
        return self.nfa.getStateByName(name)

    def getStatesByName(self, names):
        return self.nfa.getStatesByName(names)

    def getInputSymbols(self):
        return self.nfa.getInputSymbols()

    def epsClosure(self, states):

        closure_set = set(states)
        stack_states = list(states)

        while stack_states:
            state = stack_states.pop()
            for input_symbol, next_state in state.transitions:
                # To avoid infinite loop
                if next_state not in closure_set:
                    if input_symbol == "ϵ":
                        closure_set.add(next_state)
                        stack_states.append(next_state)

        closure_list = list(closure_set)
        closure_list.sort(key=lambda x: x.name)
        closure_str = ''

        for state in closure_list:
            closure_str += ' ' + state.name

        # remove the first space
        return closure_str.strip()


    def transitionStep(self, nfa, states, input_symbol):
        move_states = set()
        states_list = states.split()
        states = []

        for name in states_list:
            states.append(nfa.getStateByName(name))

        for state in states:
            for in_s, next_state in state.transitions:
                if in_s == input_symbol:
                    move_states.add(next_state)

        return move_states

    def convertNFAtoDFA(self, nfa):
        nfa_states = nfa.getStates()
        input_symbols = nfa.getInputSymbols()

        start_state = self.epsClosure([nfa_states[0]])
        self.states = {'startingState': start_state}

        queue = [start_state]
        visited_states = set([start_state])

        while queue:
            current_state = queue.pop(0)
            for in_symbol in input_symbols:
                next_states = self.epsClosure(self.transitionStep(nfa, current_state, in_symbol))

                if next_states == '' or next_states == ' ':
                    continue

                if next_states not in visited_states:
                    queue.append(next_states)
                    visited_states.add(next_states)

                if current_state not in self.states:
                    self.states[current_state] = {}
                self.states[current_state][in_symbol] = next_states

            self.states.setdefault(current_state, {})['isAcceptedState'] = nfa.isAcceptedState(nfa.getStatesByNames(current_state))

        return self.states

    def toDict(self):
        return self.states.copy()

    def visualize(self, name='output/dfa.gv', view=False):

        graph = graphviz.Digraph(engine='dot')

        for state, transitions in self.states.items():
            if state == 'startingState':
                continue
            if transitions['isAcceptedState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')

            for char, next_state in transitions.items():

                if char == 'isAcceptedState':
                    continue
                children_states = next_state.split(',')

                for child in children_states:
                    graph.edge(state, child, label=char)
        graph.render("output/dfa", view=False)


In [15]:
class minimizedDFA:
    def __init__(self, dfa):
        self.dfa = dfa
        self.states = self.dfaTOMinDFA(dfa)
    # Utility functions
    def getGroupStates(self, group):
        result = []
        for state in group:
            for key, value in state.items():
                result.append(key)
        return result

    def dfaTOMinDFA(self, dfa):
        states = dfa.toDict()
        input_symbols = dfa.getInputSymbols()

        states.pop('startingState') # no minimization for the starting state
        groups = []
        acceptance_states = []
        non_acceptance_states = []

        for key, value in states.items():
            if value["isAcceptedState"] == True:
                acceptance_states.append({key: value})
            else:
                non_acceptance_states.append({key: value})
        groups.append(acceptance_states)
        groups.append(non_acceptance_states)

        # for each group in groups, if there is a state that has a transition to a state in another group, split the group into two groups
        need_split = True
        while need_split:
            need_split = False
            for index, group in enumerate(groups):
                if not group:   # skip if empty group
                    continue

                group_indices_first_state = {}
                state = group[0] # get first state in the group
                for key, value in state.items():
                    for s in input_symbols:
                        if s in value:
                            group_indices_first_state[s] = [j for j, group in enumerate(groups) if value[s] in self.getGroupStates(group)][0]

                splitted_states = []
                for k, state in enumerate(group):
                    group_indices = {}
                    for key, value in state.items():
                        for s in input_symbols:
                            if s in value:
                                group_indices[s] = [j for j, group in enumerate(groups) if value[s] in self.getGroupStates(group)][0]

                    if group_indices != group_indices_first_state:
                        need_split = True
                        splitted_states.append(state)
                if splitted_states: # if there are states to split
                    groups.insert(index + 1, list(splitted_states))   # make a new group for the splitted states
                    groups[index] = [state for state in group if state not in splitted_states]
        return self.mergeGroupStates(groups)

    def mergeGroupStates(self, groups):
        new_states_numbers = {}

        for group_index, group in enumerate(groups):
            for state in group:
                for key, value in state.items():
                    new_states_numbers[key] = str(group_index)

        final_groups = {'startingState':0}

        for group_index, group in enumerate(groups):
            for state in group:
                for key, value in state.items():
                    for s, n_state in value.items():
                        if n_state in new_states_numbers:
                            value[s] = str(new_states_numbers[n_state])
                            final_groups[str(group_index)] = value
        return final_groups

    def toDict(self):
        return self.states

    def visualize(self, name='output/min.gv', view=False):
        graph = graphviz.Digraph(engine='dot')
        for state, transitions in self.states.items():
            if state == 'startingState':
                continue
            if transitions['isAcceptedState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')
            for char, next_state in transitions.items():
                if char == 'isAcceptedState':
                    continue
                children_states = next_state.split(',')
                for child in children_states:
                    graph.edge(state, child, label=char)
        graph.render("output/minimizedDFA", view=False)

In [56]:

cases = {
    0: '(CD(C|D)*C+)',
    1: ' (CD)',
    2: '(C|D)',
    3: '([C-Z])',
    4: '(C)+',
    5: '(C)*',
    6: '(((CD)((C|D)*))(CD))',
    7: '((((CD)|[C-Z])+)([C-Z]*))',
    8: '(((((CDE)|F)|((([C-Z])S)*))+)((CD)F))',
    9: '((([c-z_])(([c-z0-9_])*))(([!?])?))',
    10: '(C(((D*)|(DA))*))((CG)|(D([DEF])))',
    11: '(CD',
    12: '(c([c-d))',
    13: '((c|d)|)'
}

for i in range(0, 14):

  if i == 7:
    continue

  regex = cases[i]

  if not validate_regex(regex):
    print("invalid regex (index)", i)
    continue

  postfix = postfixByShuntYard(regex)
  nfa = NFA(postfix=postfix.getPostfixedRegex())
  visualize('new4/graphs' + str(i) +'/nfa.gv', nfa.JSONIFY(), view=False)
  dfa = DFA(nfa)
  visualize('new4/graphs' +  str(i) +'dfa.gv', dfa.toDict(), view=False)
  minDfa = minimizedDFA(dfa)
  visualize('new4/graphs' + str(i) +'min_dfa.gv', minDfa.toDict(), view=False)
  print("=================== Finished ", str(i))




regex = input("Enter regular expression: ")
if not validate_regex(regex):
    print("invalid regex")


postfix = postfixByShuntYard(regex)
nfa = NFA(postfix=postfix.getPostfixedRegex())
visualize('graphs3/nfa.gv', nfa.JSONIFY(), view=False)
dfa = DFA(nfa)
visualize('graphs3/dfa.gv', dfa.toDict(), view=False)
minDfa = minimizedDFA(dfa)
visualize('graphs3/min_dfa.gv', minDfa.toDict(), view=False)
print("=================== Finished =======================")



=================== Finished  0
=================== Finished  1
=================== Finished  2
=================== Finished  3
=================== Finished  4
=================== Finished  5
=================== Finished  6
=================== Finished  8
invalid regex (index) 9
=================== Finished  10
invalid regex (index) 11
invalid regex (index) 12
invalid regex (index) 13
